In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [3]:
data = pd.read_excel("data.xls", header=3)
data  = data.rename(columns = {"Unnamed: 0":"Country", "Unnamed: 1":"Goal"})
data = data.loc[data.Goal.str.strip() == "деловая"]

years = np.arange(2010,2024).astype(str)

year_indexes = [data.columns.get_loc(year) for year in years]
columns_to_drop = []

#cумма по годам
for year_index in year_indexes:
    year = data.columns[year_index]
    columns_to_sum = data.iloc[:, year_index:year_index + 4].columns
    data[year] = data[columns_to_sum].sum(axis=1, skipna=True)

    if len(columns_to_sum) > 1:
        columns_to_drop.extend(columns_to_sum[1:])
#дропаем лишние колонки
data = data.drop(columns=columns_to_drop)

data.to_csv("data_cleaned.csv")

In [4]:
data["Diff_23_18"] = data['2023']-data['2018']
data = data[data["Diff_23_18"] != 0]
data.to_csv("data_cleaned.csv")

In [16]:
df = pd.read_csv("data_cleaned.csv")
df = df.loc[:, ['country_code','Diff_23_18']]
gdf = gpd.read_file('world-administrative-boundaries/world-administrative-boundaries.shp')
new_df = gdf.merge( df, left_on = 'color_code', right_on = 'country_code')
new_df.to_file('country_plots.shp')

/tmp/ipykernel_15415/3512657041.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  new_df.to_file('country_plots.shp')
